Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.7/349.7 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4049 sha256=1015e84102b4e5fd0ebf1ed8aa384ac99167f4b54750a8fe6b73d48ba4e53fc4
  Stored in directory: /root/.cache/pip/wheels/a8/03/64/8557323821d25118c3a2dc1646996f7a962a8970d4b7d22473
Successfully built autograd-gamma


In [1]:
!pip install scikit-survival

     ---------------------------------------- 0.0/718.6 kB ? eta -:--:--
     ------ ------------------------------- 122.9/718.6 kB 2.4 MB/s eta 0:00:01
     ---------------------- --------------- 430.1/718.6 kB 4.5 MB/s eta 0:00:01
     ------------------------------ ------- 583.7/718.6 kB 4.1 MB/s eta 0:00:01
     -------------------------------------- 718.6/718.6 kB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/292.5 kB ? eta -:--:--
     ------------------ ------------------- 143.4/292.5 kB 2.8 MB/s eta 0:00:01
     --------------------------------- ---- 256.0/292.5 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 292.5/292.5 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/72.0 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/72.0 kB ? eta -:--:--
     ---------------- --------------------- 30.7/72.0 kB 660.6 kB/s eta 0:00:01
     -------------------------------- ----- 61.4/72.0 kB 656.4 kB/s e

In [2]:
# librairies
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis


In [4]:
# Librairies Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import  GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression

In [6]:
#Load Data
data= pd.read_csv("features_with_clinical_data_KNN.csv", delimiter= ',')
data=data.drop(['patient_id', 'PFS'], axis=1)
data.head(10)

,MYC IHC,BCL2 IHC,BCL6 IHC,CD10 IHC,MUM1 IHC,HANS,BCL6 FISH,MYC FISH,BCL2 FISH,Age,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,weight
0,0.0,30.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,61.0,...,0.989375,-3.942934,-0.733295,4.967747,1.334263,-0.986458,2.388044,-1.648032,-0.589221,0.004299
1,10.0,0.0,70.0,0.0,1.0,0.0,0.0,0.0,0.0,24.0,...,0.550286,-4.414318,-0.276240,4.564934,-0.723174,1.763377,4.393436,-1.429609,-1.713000,0.006865
2,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,...,2.067740,-3.714199,-0.387030,4.627776,1.488059,-0.337735,3.142957,0.683101,-1.692691,0.003299
3,10.0,42.0,18.0,0.0,0.6,0.2,0.4,0.0,0.0,64.0,...,0.584208,-3.478100,1.927194,4.956064,2.365260,-1.304836,5.289010,-0.652614,-0.085571,0.002033
4,6.0,20.0,42.0,0.2,0.4,0.2,0.0,0.0,0.0,41.0,...,0.633456,-3.469962,1.479658,3.717409,1.480196,-1.532882,0.858435,-1.163568,-0.669890,0.000833
5,0.0,80.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,78.0,...,1.221913,-4.361817,2.355842,5.300560,0.533461,-1.320051,1.984959,-0.853931,-0.846607,0.003632
6,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,...,3.487181,-5.328841,-0.580664,6.214092,1.966028,-0.664566,2.138801,0.261990,-1.470663,0.007698
7,0.0,20.0,30.0,0.0,1.0,0.0,1.0,0.0,0.0,62.0,...,0.271314,-1.673310,-0.031450,3.288475,1.381165,-1.919586,2.031429,-1.418477,0.359382,0.007465
8,10.0,10.0,50.0,1.0,1.0,1.0,0.0,0.0,0.0,54.0,...,2.532299,-6.912847,-1.156980,8.052887,0.121927,2.348103,4.989056,-0.491396,-3.756457,0.003732
9,50.0,0.0,60.0,0.0,0.0,1.0,1.0,0.0,0.0,65.0,...,0.391128,-3.949425,-0.165909,4.055010,0.211571,0.161041,3.747123,-1.198994,-1.020321,0.006665


In [20]:
feat = data.columns
feat

Index(['MYC IHC', 'BCL2 IHC', 'BCL6 IHC', 'CD10 IHC', 'MUM1 IHC', 'HANS',
       'BCL6 FISH', 'MYC FISH', 'BCL2 FISH', 'Age',
       ...
       '1015', '1016', '1017', '1018', '1019', '1020', '1021', '1022', '1023',
       'weight'],
      dtype='object', length=1045)

In [8]:
#Mis à l'échelle
pipeline = Pipeline([ 
                     ('scaler',StandardScaler()), 
                     ('model',Lasso()) 
])

In [9]:
#Approche de validation croisée
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [ ]:
from sklearn.datasets import load_diabetes
X,y = load_diabetes(return_X_y=True)
X[:10]

array([[ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
        -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
        -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, -0.00567042, -0.04559945,
        -0.03419447, -0.03235593, -0.00259226,  0.00286131, -0.02593034],
       [-0.08906294, -0.04464164, -0.01159501, -0.03665608,  0.01219057,
         0.02499059, -0.03603757,  0.03430886,  0.02268774, -0.00936191],
       [ 0.00538306, -0.04464164, -0.03638469,  0.02187239,  0.00393485,
         0.01559614,  0.00814208, -0.00259226, -0.03198764, -0.04664087],
       [-0.09269548, -0.04464164, -0.04069594, -0.01944183, -0.06899065,
        -0.07928784,  0.04127682, -0.0763945 , -0.04117617, -0.09634616],
       [-0.04547248,  0.05068012, -0.04716281, -0.01599898, -0.04009564,
        -0.02480001,  0.00077881, -0.03949338

In [7]:
# Séparer les données
Y=data[['Follow-up Status','OS']]
#Y=np.array(Y)
X=data.drop(['Follow-up Status','OS'], axis=1)

In [10]:
#Train the model
search.fit(X,Y)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-1.466 total time=   0.0s
[CV 2/5] END .................model__alpha=0.1;, score=-2.826 total time=   0.0s
[CV 3/5] END .................model__alpha=0.1;, score=-3.358 total time=   0.0s
[CV 4/5] END .................model__alpha=0.1;, score=-1.983 total time=   0.0s
[CV 5/5] END .................model__alpha=0.1;, score=-2.190 total time=   0.0s
[CV 1/5] END .................model__alpha=0.2;, score=-1.506 total time=   0.0s
[CV 2/5] END .................model__alpha=0.2;, score=-2.761 total time=   0.0s
[CV 3/5] END .................model__alpha=0.2;, score=-2.977 total time=   0.0s
[CV 4/5] END .................model__alpha=0.2;, score=-1.816 total time=   0.0s
[CV 5/5] END .................model__alpha=0.2;, score=-1.844 total time=   0.0s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-1.499 total time=   0.0s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [11]:
search.best_params_

{'model__alpha': 0.4}

In [22]:
#Valeur des coefficients de regression et leur importancd
coefficients = search.best_estimator_.named_steps['model'].coef_
importance = np.abs(coefficients)
importance2= importance[np.abs(coefficients)>0]
importance2
importance

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
# Trouver l'index de l'élément 6
index = np.where( importance == 0.00486397)
for i in importance2:
  index = np.where( importance == i)
  print(index)

(array([1], dtype=int64), array([9], dtype=int64))
(array([1], dtype=int64), array([17], dtype=int64))
(array([1], dtype=int64), array([117], dtype=int64))


In [21]:
feat[coefficients != 0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
removed_feats = X.columns[( importance == 0).ravel().tolist()]

IndexError: ignored

In [ ]:
feat=list(feat)
feat=np.array(feat)
feat[importance2>0]

IndexError: ignored

In [ ]:
# Deuxième méthode

scaler = StandardScaler()
scaler.fit(X)

StandardScaler()

In [ ]:
sel_ = SelectFromModel(Lasso(alpha=0.001, random_state=10))
sel_.fit(scaler.transform(X), Y)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.524e-01, tolerance: 3.821e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.736e+00, tolerance: 1.960e-01
  model = cd_fast.enet_coordinate_descent(


SelectFromModel(estimator=Lasso(alpha=0.001, random_state=10))

In [ ]:
sel_.get_support()


array([ True,  True,  True, ..., False,  True,  True])

In [ ]:
sel_.get_feature_names_out()

array(['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10',
       'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x20',
       'x23', 'x24', 'x25', 'x26', 'x28', 'x31', 'x34', 'x40', 'x41',
       'x44', 'x45', 'x46', 'x50', 'x54', 'x60', 'x67', 'x70', 'x72',
       'x74', 'x79', 'x82', 'x83', 'x84', 'x86', 'x87', 'x89', 'x90',
       'x95', 'x97', 'x101', 'x102', 'x104', 'x105', 'x106', 'x107',
       'x108', 'x109', 'x111', 'x112', 'x117', 'x119', 'x120', 'x125',
       'x127', 'x129', 'x132', 'x138', 'x139', 'x141', 'x142', 'x146',
       'x147', 'x150', 'x154', 'x155', 'x158', 'x161', 'x162', 'x168',
       'x169', 'x170', 'x171', 'x175', 'x176', 'x177', 'x178', 'x184',
       'x186', 'x187', 'x191', 'x196', 'x197', 'x202', 'x205', 'x206',
       'x207', 'x209', 'x210', 'x211', 'x212', 'x213', 'x215', 'x217',
       'x218', 'x219', 'x220', 'x222', 'x225', 'x234', 'x235', 'x237',
       'x240', 'x241', 'x243', 'x245', 'x246', 'x247', 'x248', 'x250',
       'x

In [ ]:
removed_feats = X.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]

IndexError: ignored

In [ ]:
X_train_selected = sel_.transform(scaler.transform(X))


AttributeError: ignored